In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import copy

Notes:

Example trading data as: 8 hours, 250 days, 6 years = 12000

Calculate the simple delta calculation

In [3]:
def return_price_dynamic_data(df_input="Data\DowJones_Daily_1Y.csv", price_string="Close*", return_plot=True, scale_to_first_value=True):

    df_close = extract_close_value(df_input=df_input, price_string=price_string, scale_to_first_value=scale_to_first_value)
    
    #calculate deltas
    df_close["delta"] = np.zeros(len(df_close.index))
    for i in df_close.index[1:]:
        df_close.loc[i, "delta"] = df_close[price_string][i - 1] - df_close[price_string][i]
    df_close["delta"][0] = df_close["delta"][1]

    #produce plots and return values
    if return_plot == True:
        fig, axs = plt.subplots(2)
        fig.suptitle('Vertically stacked subplots')
        axs[0].plot(df_close[price_string])
        axs[1].plot(df_close["delta"])
        return fig, df_close
    elif return_plot == False:
        return df_close

def extract_close_value(df_input="Data\DowJones_Daily_1Y.csv", price_string="Close*", scale_to_first_value=True):
    if isinstance(df_input, str):
        df = pd.read_csv(df_input)
    else:
        df = df_input
    
    #handle strings
    if isinstance(df[price_string][0], str):
        df[price_string] = df[price_string].str.replace(',','')
        df[price_string] = df[price_string].astype(float)
    df_close     = pd.DataFrame(df[price_string])
    
    if scale_to_first_value == True:
        scale = df_close[price_string][0]
        df_close[price_string] = df_close[price_string] / scale
    return df_close

def run_analysis_on_folder(folder_location_list=["C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Econophysics\\Analysis\\Data\\h_us_txt\\data\\hourly\\us\\nasdaq stocks\\1"], price_string="<CLOSE>", return_plot=False, scale_to_first_value=True, cap_analysis_at_file_qty=20):
    delta_table = pd.DataFrame(columns=["delta"])    
    file_count = 0
    for folder_path in folder_location_list:
        for filename in os.listdir(folder_path):
            if file_count < cap_analysis_at_file_qty:
                file = os.path.join(folder_path, filename)
                file_count += 1
                temp = return_price_dynamic_data(df_input=file, price_string=price_string, return_plot=return_plot, scale_to_first_value=scale_to_first_value)
                delta_table = pd.concat([delta_table, pd.DataFrame(temp["delta"])], ignore_index=True) 
    
    return delta_table

#delta_table = run_analysis_on_folder()
#print(len(delta_table))
#p = plt.hist(delta_table, bins=200)
    

More complex sigma(delta t) calculator

In [10]:
def return_sigma_delta_t(df_input="Data\DowJones_Daily_1Y.csv", price_string="Close*", return_plot=True, scale_to_first_value=True, delta_ts=range(1, 11)):
    sigma_delta_table = pd.DataFrame()
    df_close = extract_close_value(df_input=df_input, price_string=price_string, scale_to_first_value=scale_to_first_value)
    
    sigma_delta_t = []
    
    #calculate deltas
    for incri in delta_ts:
        #df_close[str(incri)] = np.nan(len(df_close.index))
        df_close[str(incri)] = np.nan
        for time_step in df_close.index[incri:]:
            df_close.loc[time_step, str(incri)] = df_close[price_string][time_step - incri] - df_close[price_string][time_step]
        sigma_delta_t = sigma_delta_t + [df_close[str(incri)].std()]
    
    return sigma_delta_t

time_steps_inputs=[1e0, 5e0, 1e1, 5e1, 1e2, 5e2, 1e3, 5e3, 1e4]

test = return_sigma_delta_t(time_steps=time_steps_inputs)
print(test)


TypeError: slice indices must be integers or None or have an __index__ method

In [7]:
print("Hello")

Hello
